# Chapter 4: Core Concepts You Must Get Right Early

## 4.1 The Hierarchy: Cluster, Database, Schema, Object

PostgreSQL organizes data in a four-level hierarchy that differs significantly from other database systems. Misunderstanding these boundaries leads to security vulnerabilities, backup failures, and operational complexity.

### 4.1.1 The Cluster (Instance)

A **cluster** (or instance) is a running PostgreSQL server process (`postmaster`) managing a single data directory (`PGDATA`). It contains multiple databases and listens on a single port.

**Key Constraints:**
- One cluster = One `PGDATA` directory = One port
- Global objects (roles, tablespaces, replication slots) exist at cluster level
- A connection is always to one specific database within the cluster; you cannot query across databases in the same cluster without extensions like `postgres_fdw`

```sql
-- View cluster-level information
SELECT 
    pg_postmaster_start_time() as cluster_started,
    inet_server_addr() as server_ip,
    inet_server_port() as server_port,
    current_setting('data_directory') as data_directory,
    current_setting('cluster_name') as cluster_name;  -- Set in postgresql.conf
```

**Industry Standard:** Run separate clusters for truly isolated environments (production vs. staging) rather than relying on database-level separation within one cluster. This prevents accidental cross-database queries and simplifies backup strategies.

### 4.1.2 Databases (Catalogs)

A **database** is a named collection of schemas, tables, and other objects. Each database is isolated; you cannot join tables between databases without Foreign Data Wrappers (FDW).

**Database Creation Standards:**

```sql
-- Create database with explicit encoding and locale (never rely on template1 defaults)
CREATE DATABASE appdb
    WITH 
    OWNER = app_admin
    ENCODING = 'UTF8'
    LC_COLLATE = 'en_US.UTF-8'
    LC_CTYPE = 'en_US.UTF-8'
    TEMPLATE = template0      -- Clean template, ensures no objects from template1
    CONNECTION LIMIT = 200;   -- Prevent connection exhaustion

-- Comment databases (auditing)
COMMENT ON DATABASE appdb IS 'Production application database - Owned by Platform Team';
```

**Critical Database Properties:**
- **Encoding**: Fixed at creation (`UTF8` is industry standard; never use SQL_ASCII)
- **Locale** (`LC_COLLATE`): Determines string sort order; affects index usage for `ORDER BY` and `LIKE` ranges
- **Tablespace**: Default storage location for this database's objects

```sql
-- Check database encoding and locale (critical for text comparisons)
SELECT 
    datname,
    pg_encoding_to_char(datencoding) as encoding,
    datcollate as collation,
    datctype as character_classification
FROM pg_database 
WHERE datname = current_database();

-- Database size monitoring
SELECT 
    datname,
    pg_size_pretty(pg_database_size(datname)) as size,
    datallowconn as connections_allowed,
    datconnlimit as connection_limit
FROM pg_database
ORDER BY pg_database_size(datname) DESC;
```

### 4.1.3 Schemas (Namespaces)

A **schema** is a namespace within a database that contains tables, views, functions, and other objects. Schemas prevent naming collisions and enable multi-tenancy patterns.

**The `public` Schema Problem:**

By default, PostgreSQL places objects in the `public` schema. This is dangerous because:
- All users have `CREATE` privileges on `public` by default (until PostgreSQL 15)
- It encourages dumping objects into a flat namespace
- It complicates migrations and access control

**Industry Standard Schema Strategy:**

```sql
-- 1. Revoke public create privileges (security baseline)
REVOKE CREATE ON SCHEMA public FROM PUBLIC;
REVOKE ALL ON SCHEMA public FROM PUBLIC;

-- 2. Create application-specific schema
CREATE SCHEMA IF NOT EXISTS app;
COMMENT ON SCHEMA app IS 'Application tables and business logic';

-- 3. Create isolated schemas for extensions
CREATE SCHEMA extensions;
CREATE EXTENSION IF NOT EXISTS pgcrypto WITH SCHEMA extensions;

-- 4. Migration/schema management schema
CREATE SCHEMA ddl_history;

-- 5. Set search path (schema resolution order)
-- NEVER include 'public' first; explicitly order schemas
ALTER DATABASE appdb SET search_path TO app, extensions, public;

-- Verify
SHOW search_path;
-- Output: app, extensions, public
```

**Search Path Resolution:**

```sql
-- PostgreSQL resolves unqualified names using search_path
SELECT * FROM users;  -- Finds first 'users' table in search_path order

-- Explicit schema qualification (production code standard)
SELECT * FROM app.users;

-- Current schemas in search path
SELECT current_schemas(true);  -- Includes implicit schemas
SELECT current_schemas(false); -- User-defined only
```

### 4.1.4 Objects (Tables, Indexes, Views, etc.)

The final level contains the actual data structures. Object names must be unique within their schema.

**Fully Qualified Names:**
```sql
-- Format: database.schema.object
-- Note: You cannot specify database in queries (use FDW for cross-db queries)
SELECT * FROM appdb.app.users;  -- ERROR: cross-database references not implemented

-- Valid: schema.object
SELECT * FROM app.users;
```

---

## 4.2 System Catalogs: `pg_catalog` vs `information_schema`

PostgreSQL provides two metadata interfaces: the native `pg_catalog` and the SQL-standard `information_schema`. Understanding when to use each is essential for portable tooling and deep inspection.

### 4.2.1 `pg_catalog` (The Native Interface)

`pg_catalog` contains the actual system tables that PostgreSQL uses internally. It is optimized for PostgreSQL-specific features and performance.

**When to Use `pg_catalog`:**
- Performance-critical metadata queries
- Access to PostgreSQL-specific features (tablespaces, TOAST, vacuum stats, replication slots)
- Administrative scripts that won't port to other databases anyway

```sql
-- List tables with PostgreSQL-specific details (OID, tablespace, persistence)
SELECT 
    c.oid,
    n.nspname as schema,
    c.relname as table,
    pg_size_pretty(pg_table_size(c.oid)) as table_size,
    pg_size_pretty(pg_indexes_size(c.oid)) as indexes_size,
    t.spcname as tablespace,
    CASE c.relpersistence 
        WHEN 'p' THEN 'permanent'
        WHEN 'u' THEN 'unlogged'
        WHEN 't' THEN 'temporary'
    END as persistence,
    c.relhasindex as has_indexes,
    c.reltuples::bigint as estimated_rows
FROM pg_class c
JOIN pg_namespace n ON n.oid = c.relnamespace
LEFT JOIN pg_tablespace t ON t.oid = c.reltablespace
WHERE c.relkind = 'r'  -- Ordinary tables only
  AND n.nspname NOT IN ('pg_catalog', 'information_schema')
ORDER BY pg_table_size(c.oid) DESC;
```

### 4.2.2 `information_schema` (The SQL Standard)

`information_schema` provides a standardized view of metadata defined by the SQL specification (ISO/IEC 9075). It is portable across database systems but lacks PostgreSQL-specific details.

**When to Use `information_schema`:**
- Writing portable SQL generation tools
- ORM introspection queries
- Compliance reporting requiring standard SQL structures

```sql
-- Standard SQL table listing (works on PostgreSQL, MySQL, SQL Server, etc.)
SELECT 
    table_schema,
    table_name,
    table_type
FROM information_schema.tables
WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
  AND table_type = 'BASE TABLE'
ORDER BY table_schema, table_name;

-- Standard column inspection
SELECT 
    table_schema,
    table_name,
    column_name,
    data_type,
    is_nullable,
    column_default
FROM information_schema.columns
WHERE table_schema = 'app'
  AND table_name = 'users'
ORDER BY ordinal_position;
```

### 4.2.3 Key Catalog Queries for Operations

**Active Connections and Locks:**
```sql
-- PostgreSQL-native (pg_catalog)
SELECT 
    datname,
    pid,
    usename,
    application_name,
    client_addr,
    state,
    wait_event_type,
    wait_event,
    query_start,
    state_change,
    left(query, 100) as query_preview
FROM pg_stat_activity
WHERE state != 'idle'
ORDER BY query_start;

-- Lock conflicts (pg_catalog only)
SELECT 
    blocked_locks.pid AS blocked_pid,
    blocked_activity.usename AS blocked_user,
    blocking_locks.pid AS blocking_pid,
    blocking_activity.usename AS blocking_user,
    blocked_activity.query AS blocked_statement,
    blocking_activity.query AS blocking_statement
FROM pg_catalog.pg_locks blocked_locks
JOIN pg_catalog.pg_stat_activity blocked_activity ON blocked_activity.pid = blocked_locks.pid
JOIN pg_catalog.pg_locks blocking_locks ON blocking_locks.locktype = blocked_locks.locktype
    AND blocking_locks.database = blocked_locks.database
    AND blocking_locks.relation = blocked_locks.relation
    AND blocking_locks.page = blocked_locks.page
    AND blocking_locks.tuple = blocked_locks.tuple
    AND blocking_locks.virtualxid = blocked_locks.virtualxid
    AND blocking_locks.transactionid = blocked_locks.transactionid
    AND blocking_locks.classid = blocked_locks.classid
    AND blocking_locks.objid = blocked_locks.objid
    AND blocking_locks.objsubid = blocked_locks.objsubid
    AND blocking_locks.pid != blocked_locks.pid
JOIN pg_catalog.pg_stat_activity blocking_activity ON blocking_activity.pid = blocking_locks.pid
WHERE NOT blocked_locks.granted;
```

**Index Usage Statistics:**
```sql
-- Native catalog query for index utilization
SELECT 
    schemaname,
    tablename,
    indexname,
    idx_scan,           -- Number of times index was used
    idx_tup_read,
    idx_tup_fetch,
    pg_size_pretty(pg_relation_size(indexrelid)) as index_size
FROM pg_stat_user_indexes
ORDER BY idx_scan DESC;
```

---

## 4.3 Physical Storage: Pages, TOAST, and Visibility

Understanding how PostgreSQL physically stores data prevents design errors that cause bloat, performance degradation, and vacuum issues.

### 4.3.1 The Page (Block) Structure

PostgreSQL stores table data in **pages** (also called blocks) of 8KB by default (compile-time constant, rarely changed).

**Page Layout:**
- **Header** (24 bytes): Page metadata (checksum, free space pointers)
- **Line Pointers** (4 bytes each): Offsets to actual tuples
- **Free Space**: Available for new tuples or enlarged existing tuples
- **Tuples** (Heap Tuples): The actual row data with headers
- **Special Space**: Index-specific data (for index pages)

**Implications:**
- Row data cannot exceed ~8KB (minus overhead) without TOAST
- Updating a row creates a new tuple version in the same or different page
- Sequential scans read entire pages, not individual rows

```sql
-- View page-level storage parameters
SELECT 
    relname,
    relpages,           -- Estimated pages (updated by vacuum/analyze)
    reltuples,          -- Estimated rows
    pg_relation_size(oid) as total_bytes,
    pg_size_pretty(pg_relation_size(oid)) as size
FROM pg_class
WHERE relname = 'users';

-- Page inspection extension (for deep debugging)
CREATE EXTENSION IF NOT EXISTS pageinspect;

-- View raw page data (superuser only)
SELECT * FROM heap_page_items(get_raw_page('users', 0));
```

### 4.3.2 TOAST (The Oversized-Attribute Storage Technique)

TOAST automatically stores large column values (typically >2KB) in a separate table, storing only a pointer in the main table.

**When TOAST Activates:**
- Row size exceeds ~2KB (1/4 of page size)
- Columns use TOAST-able types: `TEXT`, `BYTEA`, `JSONB`, `ARRAY`, etc.
- Numeric types and short strings are never TOASTed

**TOAST Strategies (Per-Column):**
- **PLAIN**: No TOAST (for small fixed-width types like `INTEGER`)
- **EXTENDED**: Compress if possible, then store externally (default for `TEXT`, `BYTEA`)
- **EXTERNAL**: Store externally without compression (faster for data already compressed like JPEG)
- **MAIN**: Compress inline, store externally only if no other option

```sql
-- Check TOAST strategy for columns
SELECT 
    a.attname as column_name,
    pg_catalog.format_type(a.atttypid, a.atttypmod) as data_type,
    CASE a.attstorage 
        WHEN 'p' THEN 'PLAIN (no TOAST)'
        WHEN 'e' THEN 'EXTERNAL (no compression)'
        WHEN 'x' THEN 'EXTENDED (compress then toast)'
        WHEN 'm' THEN 'MAIN (compress first)'
    END as toast_strategy,
    pg_column_size(a.*) as column_size_bytes
FROM pg_attribute a
JOIN pg_class c ON c.oid = a.attrelid
JOIN pg_namespace n ON n.oid = c.relnamespace
WHERE c.relname = 'large_documents'
  AND a.attnum > 0
  AND NOT a.attisdropped;

-- Set TOAST strategy for specific columns (e.g., pre-compressed data)
ALTER TABLE documents 
    ALTER COLUMN image_data SET STORAGE EXTERNAL;  -- Don't compress JPEGs again

-- Check TOAST table size (bloat indicator)
SELECT 
    relname as main_table,
    pg_size_pretty(pg_relation_size(oid)) as main_size,
    pg_size_pretty(pg_relation_size(reltoastrelid)) as toast_size
FROM pg_class
WHERE relname = 'large_table';
```

### 4.3.3 The Visibility Map and Free Space Map

**Visibility Map (VM):**
- One bit per page indicating if all tuples are visible to all transactions
- Enables **index-only scans** (reading index without touching heap)
- Maintained by `VACUUM`

**Free Space Map (FSM):**
- Tracks available space in pages for new/updated tuples
- Prevents scanning pages that cannot fit new data
- Stored in `_fsm` files alongside table files

```sql
-- Check if index-only scans are possible (visibility map coverage)
SELECT 
    schemaname,
    tablename,
    attname,
    n_tup_read,         -- Heap reads
    n_tup_fetch,        -- Index fetches
    n_tup_hot_upd_err,  -- HOT update failures
    vacuum_count,
    last_vacuum,
    last_autovacuum
FROM pg_stat_user_tables
WHERE tablename = 'users';
```

---

## 4.4 Naming Conventions and SQL Style Standards

Consistent naming prevents ambiguity, simplifies automated tooling, and reduces cognitive load during incident response.

### 4.4.1 Case Sensitivity and Quoting

PostgreSQL folds unquoted identifiers to lowercase (SQL standard behavior). **Never use quoted identifiers** (`"MixedCase"`) in production code.

```sql
-- These are identical (folded to lowercase)
CREATE TABLE UserAccounts (id int);
CREATE TABLE useraccounts (id int);
CREATE TABLE USERACCOUNTS (id int);

-- Quoted identifiers create case-sensitive names (AVOID)
CREATE TABLE "UserAccounts" (id int);  -- Must quote forever after
SELECT * FROM UserAccounts;  -- ERROR: relation "useraccounts" does not exist
SELECT * FROM "UserAccounts";  -- Required forever
```

### 4.4.2 Industry Standard Naming Rules

**General Principles:**
- **snake_case** for all identifiers (tables, columns, constraints, functions)
- **Singular nouns** for table names (`user`, not `users`; `order`, not `orders`)
- **Avoid reserved words** (`order`, `user`, `group` require quoting; use `user_account`, `sales_order`)
- **Explicit constraint names** for debugging and migrations

**Tables:**
```sql
-- Good
CREATE TABLE user_account (
    user_id BIGINT PRIMARY KEY,
    email_address VARCHAR(255) NOT NULL,
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW()
);

-- Bad (plural, reserved word, camelCase)
CREATE TABLE "Users" (
    userID int,
    "EmailAddress" varchar(255)
);
```

**Columns:**
```sql
-- Standard suffixes for semantic meaning
-- _id: Primary or foreign keys
-- _at: Timestamp columns (created_at, updated_at, deleted_at)
-- _by: Actor references (created_by, approved_by)
-- _count: Cached counts
-- _flag: Boolean indicators (is_active_flag vs is_active)
-- _pkey, _ukey, _fkey, _check: Constraint name suffixes (in constraint definitions)

CREATE TABLE sales_order (
    sales_order_id BIGSERIAL PRIMARY KEY,
    customer_id BIGINT NOT NULL REFERENCES customer(customer_id),
    order_status_code VARCHAR(20) NOT NULL,
    total_amount_cents INTEGER NOT NULL CHECK (total_amount_cents >= 0),
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    created_by_user_id BIGINT REFERENCES user_account(user_id),
    cancelled_at TIMESTAMPTZ,
    
    CONSTRAINT sales_order_status_check 
        CHECK (order_status_code IN ('pending', 'confirmed', 'shipped', 'cancelled'))
);
```

**Indexes:**
```sql
-- Naming: {table}_{column(s)}_{type}
CREATE INDEX user_account_email_address_lower_ukey 
    ON user_account (LOWER(email_address));  -- Case-insensitive unique lookup

CREATE INDEX sales_order_created_at_brin 
    ON sales_order USING BRIN (created_at);  -- Block Range Index for time-series
```

**Functions and Triggers:**
```sql
-- Functions: verb_noun or action_object
CREATE FUNCTION update_updated_at_column()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = NOW();
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Triggers: {table}_{action}_{timing}_trg
CREATE TRIGGER user_account_updated_at_before_update_trg
    BEFORE UPDATE ON user_account
    FOR EACH ROW
    EXECUTE FUNCTION update_updated_at_column();
```

### 4.4.3 Collation and Locale Considerations

Collation determines string sort order and comparison behavior. It is set at database creation and inherited by columns, but can be overridden per column or query.

```sql
-- Database default collation
SELECT datcollate FROM pg_database WHERE datname = current_database();

-- Column-specific collation (for case-insensitive sorting)
CREATE TABLE case_insensitive_lookup (
    code VARCHAR(50) COLLATE "C",           -- Binary/ASCII sort (fast, strict)
    name VARCHAR(255) COLLATE "en_US.utf8"  -- Locale-aware sort (correct for display)
);

-- Query-level collation override
SELECT * FROM names ORDER BY last_name COLLATE "C";
```

**Industry Standard:** Use `C` or `POSIX` collation for identifiers and codes (predictable, fast); use locale-specific collation for display names only.

---

## Chapter Summary

In this chapter, you learned:

1. **The Four-Level Hierarchy**: Cluster (instance) → Database (isolated catalog) → Schema (namespace) → Object (table/index). Connections are to one database; cross-database queries require Foreign Data Wrappers.
2. **Schema Strategy**: Never use `public` schema for application objects; create dedicated schemas (`app`, `extensions`) and explicitly set `search_path` to prevent squatting and enable logical separation.
3. **System Catalogs**: `pg_catalog` for PostgreSQL-specific, performance-critical metadata (OIDs, tablespaces, vacuum stats); `information_schema` for portable, standards-compliant tooling.
4. **Physical Storage**: 8KB pages (blocks) store tuples; TOAST handles values >2KB via external storage with compression strategies (`EXTENDED`, `EXTERNAL`, `MAIN`); Visibility Maps enable index-only scans.
5. **Naming Standards**: snake_case, singular table names, avoid reserved words, explicit constraint names (`{table}_{column}_{type}`), standard timestamp suffixes (`_at`, `_by`).

**Next:** In Chapter 5, we will examine PostgreSQL's data type system in depth—covering numeric precision pitfalls, text encoding and collation, timestamp handling with time zones, JSONB versus relational modeling, and array/range types—ensuring you choose the correct type for data integrity and performance.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='3. first_steps_with_psql.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='4. core_concepts_you_must_get_right_early.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
